In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from itertools import combinations
from scipy.stats import fisher_exact
import statsmodels.stats.contingency_tables as ct

In [ ]:
#read file TableObe65plus and assign to object
icd = pd.read_csv('tblICD10.csv', header=None) #, dtype={0: str, 1: str, 2: str, 3: str, 4: str}

In [ ]:
#specify columns 
icd.columns = ['patient', 'ICD', 'Hispanic', 'year']

In [ ]:
icd.head()

In [ ]:
#format the data
icd['ICD'] = icd['ICD'].str.replace('.*:', '', regex=True)
icd['ICD'] = icd['ICD'].str.replace('\..*', '', regex=True)
icd['Hispanic'] = icd['Hispanic'].str.replace('.*:', '', regex=True)

In [ ]:
icd.head()

## Hipanics Risk Analysis

In [ ]:
#estimate number of patients with each code and with each pair of disorders to later build the network
#estimate all the possible pairs of codes by selecting all the F codes

from itertools import combinations

fCodes = icd['ICD'].unique()

fCodesPairs = pd.DataFrame(list(combinations(fCodes, 2))).T

fCodesPairs = fCodesPairs.transpose()

fCodesPairs.head()

In [ ]:
#create a subset of icd only for hispanics

icd_hispanics = icd[icd['Hispanic'] == 'Y']
icd_hispanics.head(10)

In [ ]:
icd_hispanics.to_csv('icd_hispanics.txt', index=False, header=True)

In [ ]:
# create an empty data frame to save the results 

column_names = ["disAcode", "disBcode", "disA", "disB", "AB", "AnotB", "BnotA", 
                "notAnotB", "fisher", "oddsRatio", "95%confidenceInterval", "relativeRisk"]
results = pd.DataFrame(columns=column_names)

results.head()

In [ ]:
## create a data.frame with  pairs of diseases and different counts
#"disAcode", "disBcode", "disA", "disB", "AB", "AnotB", "BnotA", "notAnotB", "fisher", "oddsRatio", "95%CI","RR"

from scipy.stats import fisher_exact
import statsmodels.stats.contingency_tables as ct

results = pd.DataFrame(columns=["disAcode", "disBcode", "disA", "disB", "AB", 
                                "AnotB", "BnotA", "notAnotB", "fisher", "oddsRatio", "95%confidenceInterval", "relativeRisk"])

for i in range(fCodesPairs.shape[0]):
    code1 = fCodesPairs.iloc[i, 0]
    code2 = fCodesPairs.iloc[i, 1]
    
    dis1 = icd_hispanics[icd_hispanics['ICD'] == code1]
    dis2 = icd_hispanics[icd_hispanics['ICD'] == code2]
    
    dis12 = dis2[dis2['patient'].isin(dis1['patient'])]
    
    disAcode = code1
    disBcode = code2
    disA = len(dis1['patient'].unique())
    disB = len(dis2['patient'].unique())
    AB = len(dis12['patient'].unique())
    AnotB = disA - AB
    BnotA = disB - AB
    notAB = len(icd_hispanics['patient'].unique()) - AB - AnotB - BnotA
    
    mm = [[AB, AnotB], [BnotA, notAB]]

    try:
        _, p_value = fisher_exact(mm)
        ci_lower, ci_upper = ct.Table2x2(mm).oddsratio_confint()
        
    except Exception as e:
        print(str(e))
        print("code1:", code1, "- code2:", code2)

    conf_interval = f"({round(ci_lower, 3)}, {round(ci_upper, 3)})"
    relativeRisk = (float(AB) * len(icd_hispanics['patient'].unique())) / (float(disA) * float(disB))
    oddsRatio = (float(AB) * float(notAB)) / (float(AnotB) * float(BnotA))
    
    results.loc[i] = [disAcode, disBcode, disA, disB, AB, AnotB, BnotA,
                      notAB, p_value, oddsRatio, conf_interval, relativeRisk]


In [ ]:
results.head()

In [ ]:
# correct for multiple testing and add the comorbidity score estimation 

results['expect'] = (results['disA'].astype(float) * results['disB'].astype(float)) / len(icd_hispanics['patient'].unique())
results['score'] = np.log2((results['AB'].astype(float) + 1) / (results['expect'] + 1))
results = results.sort_values(by='fisher')

def p_adjust(p_values, method='bonferroni', n=1):
    if method == 'bonferroni':
        return np.minimum(p_values * n, 1)
     
    return p_values

results['correctedPvalue'] = pd.Series(p_adjust(results['fisher'].astype(float), method='bonferroni', n=results.shape[0]))

pd.set_option('display.width', 100)

results.head()

In [ ]:
#New
results_filt = results[results['correctedPvalue'] < 0.05]
results_filt.to_csv('results_filt.txt', index=False, header=True)

In [ ]:
results_filt.head()

In [ ]:
results_filt.shape

## Non-Hipanics Risk Analysis

In [ ]:
#create a subset of icd only for non hispanics

icd_non_hispanics = icd[icd['Hispanic'] == 'N']
icd_non_hispanics.head(10)

In [ ]:
# create an empty data frame to save the results 

column_names = ["disAcode", "disBcode", "disA", "disB", "AB", "AnotB", "BnotA", 
                "notAnotB", "fisher", "oddsRatio", "95%confidenceInterval", "relativeRisk"]
results2 = pd.DataFrame(columns=column_names)

results2.head()

In [ ]:
## create a data frame with  pairs of diseases and different counts
#"disAcode", "disBcode", "disA", "disB", "AB", "AnotB", "BnotA", "notAnotB", "fisher", "oddsRatio", "95%CI","RR"

results2 = pd.DataFrame(columns=["disAcode", "disBcode", "disA", "disB", "AB", 
                                "AnotB", "BnotA", "notAnotB", "fisher", "oddsRatio", "95%confidenceInterval", "relativeRisk"])

for i in range(fCodesPairs.shape[0]):
    code1 = fCodesPairs.iloc[i, 0]
    code2 = fCodesPairs.iloc[i, 1]
    
    dis1 = icd_non_hispanics[icd_non_hispanics['ICD'] == code1]
    dis2 = icd_non_hispanics[icd_non_hispanics['ICD'] == code2]
    
    dis12 = dis2[dis2['patient'].isin(dis1['patient'])]
    
    disAcode = code1
    disBcode = code2
    disA = len(dis1['patient'].unique())
    disB = len(dis2['patient'].unique())
    AB = len(dis12['patient'].unique())
    AnotB = disA - AB
    BnotA = disB - AB
    notAB = len(icd_non_hispanics['patient'].unique()) - AB - AnotB - BnotA
    
    mm = [[AB, AnotB], [BnotA, notAB]]
    try:
        _, p_value = fisher_exact(mm)
        ci_lower, ci_upper = ct.Table2x2(mm).oddsratio_confint()
        
    except Exception as e:
        print(str(e))
        print("code1:", code1, "- code2:", code2)

    conf_interval = f"({round(ci_lower, 3)}, {round(ci_upper, 3)})"
    relativeRisk = (float(AB) * len(icd_non_hispanics['patient'].unique())) / (float(disA) * float(disB))
    oddsRatio = (float(AB) * float(notAB)) / (float(AnotB) * float(BnotA))
    
    results2.loc[i] = [disAcode, disBcode, disA, disB, AB, AnotB, BnotA, 
                         notAB, p_value, oddsRatio, conf_interval, relativeRisk]


In [ ]:
results2.head()

In [ ]:
# correct for multiple testing and add the comorbidity score estimation 

results2['expect'] = (results2['disA'].astype(float) * results2['disB'].astype(float)) / len(icd_hispanics['patient'].unique())
results2['score'] = np.log2((results2['AB'].astype(float) + 1) / (results2['expect'] + 1))
results2 = results2.sort_values(by='fisher')

def p_adjust(p_values, method='bonferroni', n=1):
    if method == 'bonferroni':
        return np.minimum(p_values * n, 1)
     
    return p_values

results2['correctedPvalue'] = pd.Series(p_adjust(results2['fisher'].astype(float), method='bonferroni', n=results2.shape[0]))

pd.set_option('display.width', 100)

results2.head()


In [ ]:
#New
results_filt_2 = results2[results2['correctedPvalue'] < 0.05]
results_filt_2.to_csv('results_filt_2.txt', index=False, header=True)

In [ ]:
results_filt_2